<a href="https://colab.research.google.com/github/hnguyen8/API_youtube_video_statistics_Good_Mythical_Morning/blob/main/import_df_to_cloud_database_yt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#!pip install psycopg2
import psycopg2 as ps
import pandas as pd

# Mount Drive to access csv file
from google.colab import drive
drive.mount('/content/gdrive')

In [91]:
# Read csv file
path = '/content/gdrive/My Drive/DataScience/good_mythical_morning_videos.csv'

df = pd.read_csv(path)
df.head()

,Unnamed: 0,video_id,video_title,upload_date,view_count,like_count,favorite_count,comment_count
0,0,w42iXLlQK1Q,How Fast Do Snacks Go Stale?,2022-10-21,753455,24681,0,1111
1,1,BH8_FkQ3ct4,Who Can Start A Fire Faster? (Challenge),2022-10-20,710157,27646,0,1418
2,2,c4EUhTEojxo,Exotic Fruits Taste Test,2022-10-19,993281,32335,0,1889
3,3,LZZbg9AVtzs,Testing Discontinued Products,2022-10-18,743133,25272,0,1046
4,4,g_1vhq8nMA4,Frozen vs. Fast vs. Fancy Food Taste Test,2022-10-17,1260719,40417,0,1717


Define necessary functions

In [25]:
def connect_to_db(host_name, dbname, port, username, password):
    try:
        conn = ps.connect(host=host_name, database=dbname, user=username, password=password, port=port)

    except ps.OperationalError as e:
        raise e
    else:
        print('Connected!')
        return conn

In [83]:
## Create table
def create_table(curr):
  create_table_command = (
      """CREATE TABLE IF NOT EXISTS videos(
        video_id VARCHAR(255) PRIMARY KEY,
        video_title TEXT NOT NULL,
        upload_date DATE NOT NULL DEFAULT CURRENT_DATE,
        view_count INTEGER NOT NULL,
        like_count INTEGER NOT NULL,
        favorite_count INTEGER NOT NULL,
        comment_count INTEGER NOT NULL
      );
      """ )
  curr.execute(create_table_command)

In [86]:
# function to check if a video already exists
def check_if_video_exists(curr,video_id):
  query = (""" SELECT video_id FROM videos WHERE video_id = %s """)
  curr.execute(query, (video_id,))

  return curr.fetchone() is not None

In [72]:
# function to update counts for an existing video
def update_row(curr,video_id, video_title, upload_date, view_count, like_count, favorite_count, comment_count):
  query = (""" UPDATE videos
              SET 
                  video_title = %s,
                  upload_date = %s,
                  view_count = %s,
                  like_count = %s,
                  favorite_count = %s,
                  comment_count = %s
              WHERE video_id = %s;
          """)
  vars_to_update = (video_title, upload_date, view_count, like_count, favorite_count, comment_count,video_id)
  curr.execute(query, vars_to_update)
  

In [73]:
def update_db(curr, df):
  tmp_df = pd.DataFrame(columns = ['video_id', 'video_title','upload_date', 
                                  'view_count', 'like_count', 'favorite_count', 'comment_count'])

  for i, row in df.iterrows():
    if check_if_video_exists(curr,row['video_id']): # video exists, update new counts for view, like...
      update_row(curr,row['video_id'], row['video_title'], row['upload_date'], row['view_count'], row['like_count'], row['favorite_count'], row['comment_count'])
    else:     # The video doesn't exist, we append to df table.
      tmp_df = tmp_df.append(row)
  return tmp_df

In [74]:
## insert command 
def insert_into_table(curr, video_id, video_title, upload_date, view_count, like_count, favorite_count, comment_count):
  insert_into_videos = (""" INSERT INTO videos (video_id, video_title, upload_date, view_count, like_count, favorite_count, comment_count)
                            VALUES (%s, %s, %s, %s, %s, %s, %s); """)
  row_to_insert = (video_id, video_title, upload_date, view_count, like_count, favorite_count, comment_count)
  curr.execute(insert_into_videos, row_to_insert)

In [75]:
def append_from_df_to_db(curr, df):
  for i, row in df.iterrows():
    insert_into_table(curr, row['video_id'], row['video_title'], row['upload_date'], row['view_count'], row['like_count'], row['favorite_count'], row['comment_count'])


Main execution

In [82]:
## Connect to AWS Postgres database 

host_name = 'xxx'
dbname = 'xxx'
port = 'xxx'
username = 'xxx' 
password =  'xxx'
conn = None

## Make sure on AWS: 
## Publicly accessible
## EC2: Create new security group Inbound for 'all traffic' 'anywhere..'
## Modify database: change to new security group
conn = connect_to_db(host_name, dbname,port, username, password)

Connected!


In [84]:
## Create the cursor object
curr = conn.cursor()
## excecute sql command to create table
create_table(curr)  # if table already exists, this will do nothing
## commit changes
conn.commit()

In [87]:
## Create temporary dataframe holding new rows (new videos), at the same time update cloud dataframe (exisiting videos) 
new_vid_df = update_db(curr, df)
new_vid_df.head()

In [88]:
## Append temporary dataframe (new videos) to cloud dataframe (exisiting videos) 
append_from_df_to_db(curr,new_vid_df)

In [89]:
## Commit all changes
conn.commit()

In [92]:
## Close the connection to AWS
conn.close()